# Comparison of the results of the stable diffusion quantization

In [1]:
from optimum.intel.openvino import OVStableDiffusionPipeline
from diffusers.training_utils import set_seed
from IPython.display import display

/home/alex/virt_envs/stable_diffusion/lib/python3.8/site-packages/openvino/offline_transformations/__init__.py:10: FutureWarning: The module is private and following namespace `offline_transformations` will be removed in the future, use `openvino.runtime.passes` instead!
  warnings.warn(


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


ImportError: cannot import name 'set_seed' from 'diffusers' (/home/alex/virt_envs/stable_diffusion/lib/python3.8/site-packages/diffusers/__init__.py)

## Run the original pipeline
This pipeline was fine-tuned on the public [dataset](https://huggingface.co/datasets/lambdalabs/pokemon-blip-captions) with Pokemon images and the correspoinding captions. You can find the source model and the description [here](https://huggingface.co/svjack/Stable-Diffusion-Pokemon-en).

In [ ]:
pipe = OVStableDiffusionPipeline.from_pretrained("OpenVINO/stable-diffusion-pokemons-fp32", compile=False)
pipe.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)

pipe.compile()

In [ ]:
# Let's fix the seed for reproducibility.
set_seed(42)

prompt = "cartoon bird"
output = pipe(prompt, num_inference_steps=50, output_type="pil")
display(output.images[0])

## Run the quantized pipeline
Now we run the quantized pipeline that was obtained with Quantization-Aware Training on the same dataset. The original model was used as a baseline for quantization. The resulted model can be found [here](https://huggingface.co/OpenVINO/Stable-Diffusion-Pokemon-en-quantized).

In [ ]:
quantized_pipe = OVStableDiffusionPipeline.from_pretrained("OpenVINO/Stable-Diffusion-Pokemon-en-quantized", compile=False)
quantized_pipe.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)
quantized_pipe.compile()

In [ ]:
# Use the same seed to compare
set_seed(42)

output = quantized_pipe(prompt, num_inference_steps=50, output_type="pil")
display(output.images[0])

Now you can see the difference of the difference in the results and the time required to generate the image.